<a href="https://colab.research.google.com/github/Nielk74/ia-clothes/blob/master/Dataset_processing.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Présentation

Ce notebook permet d'extraire la couleur de peau, la couleur du haut et la couleur du bas de photos de mannequin. Le jeu de données utilisé ici provient de [DeepFashion-MultiModal](https://github.com/yumingj/DeepFashion-MultiModal).

In [ ]:
!pip install transformers
!pip install torch
!pip install extcolors
!pip install gdown

In [ ]:
from transformers import SegformerImageProcessor, AutoModelForSemanticSegmentation
from PIL import Image
import requests
import matplotlib.pyplot as plt
import torch.nn as nn
import os
import extcolors
import numpy as np
import tensorflow as tf
import cv2
from google.colab.patches import cv2_imshow
from sklearn.cluster import KMeans
import torch

In [ ]:
# Datasets
!gdown https://drive.google.com/uc?id=143EoZGYSm4Z1N7MRhzI9KYEwsmr8SqzG
!gdown https://drive.google.com/uc?id=1gVY3pbeZ2TIGUnkDDJ0sxGk4ZIUNjZnD
!gdown https://drive.google.com/uc?id=1Yjty0Bo8e65GkamLZIbwWdzGb-wMiX-y
!gdown https://drive.google.com/uc?id=1q7NLoZYRbO4Mb0-HorYVdLdfaOh1RY6c
!unzip -q -o women_compressed.zip
!unzip -q -o men_compressed.zip
!unzip -q -o style_du_monde_2020_2023.zip
!unzip -q -o style_du_monde_2008_2019.zip

Downloading...
From: https://drive.google.com/uc?id=143EoZGYSm4Z1N7MRhzI9KYEwsmr8SqzG
To: /content/women_compressed.zip
 28% 160M/563M [00:04<00:10, 40.2MB/s]Traceback (most recent call last):
  File "/usr/local/bin/gdown", line 8, in <module>
    sys.exit(main())
  File "/usr/local/lib/python3.10/dist-packages/gdown/cli.py", line 151, in main
  File "/usr/local/lib/python3.10/dist-packages/gdown/download.py", line 274, in download
    for chunk in res.iter_content(chunk_size=CHUNK_SIZE):
  File "/usr/local/lib/python3.10/dist-packages/requests/models.py", line 816, in generate
    yield from self.raw.stream(chunk_size, decode_content=True)
  File "/usr/local/lib/python3.10/dist-packages/urllib3/response.py", line 936, in stream
    data = self.read(amt=amt, decode_content=decode_content)
  File "/usr/local/lib/python3.10/dist-packages/urllib3/response.py", line 843, in read
    def read(
KeyboardInterrupt
 30% 167M/563M [00:04<00:10, 38.0MB/s]
^C
Downloading...
From: https://drive.goo

In [ ]:
def getColorDominantFromMask(image, mask):
  # apply mask
  binary_mask = (mask * 255).astype(np.uint8)

  # convert PIL image to openCV image
  imageCV = cv2.cvtColor(np.array(image), cv2.COLOR_RGB2BGRA)

  # add the forth dimension (opacity)
  four_channel_mask = cv2.merge([binary_mask] * 4)

  # Apply mask
  result = cv2.bitwise_and(imageCV, four_channel_mask)

  # Convert to PIL image
  pil_image = Image.fromarray(cv2.cvtColor(result, cv2.COLOR_BGR2RGBA))

  # get dominant color
  colors, pixel_count = extcolors.extract_from_image(pil_image)
  print(colors[0][0])
  return colors[0][0]

def get_colors_men(image):
  inputs = processor(images=image, return_tensors="pt").to(device)
  outputs = model(**inputs)
  logits = outputs.logits.cpu()

  upsampled_logits = nn.functional.interpolate(
      logits,
      size=image.size[::-1],
      mode="bilinear",
      align_corners=False,
  )

  # Get the segmentation prediction
  pred_seg = upsampled_logits.argmax(dim=1)[0]

  rows, cols = pred_seg.shape
  upper_mask = np.full((rows, cols), False, dtype=bool)
  lower_mask = np.full((rows,cols), False, dtype=bool)
  skin_mask = np.full((rows,cols), False, dtype=bool)

  has_upper = False
  has_lower = False
  has_skin = False
  # Iterate through the 2D tensor array with indices

  # Convert pytorch tensor to numpy array to optimize
  segmentation = pred_seg.detach().cpu().numpy()

  for i in range(rows):
      for j in range(cols):
        v = segmentation[i, j]
        if v == 4: # upper clothes
          upper_mask[i,j] = True
          has_upper = True
        elif v == 6: # pants
          lower_mask[i,j]=  True
          has_lower = True
        elif v in [11,12,13,14,15]: #body parts
          skin_mask[i,j]=  True
          has_skin = True

  if (not has_upper or not has_lower or not has_skin):
    return None
  return (getColorDominantFromMask(image, upper_mask),getColorDominantFromMask(image, lower_mask), getColorDominantFromMask(image, skin_mask))

def get_colors_women(image):
  inputs = processor(images=image, return_tensors="pt").to(device)
  outputs = model(**inputs)
  logits = outputs.logits.cpu()

  upsampled_logits = nn.functional.interpolate(
      logits,
      size=image.size[::-1],
      mode="bilinear",
      align_corners=False,
  )

  # Get the segmentation prediction
  pred_seg = upsampled_logits.argmax(dim=1)[0]

  rows, cols = pred_seg.shape
  upper_mask = np.full((rows, cols), False, dtype=bool)
  lower_mask = np.full((rows,cols), False, dtype=bool)
  skin_mask = np.full((rows,cols), False, dtype=bool)
  dress_mask = np.full((rows,cols), False, dtype=bool)
  has_upper = False
  has_lower = False
  has_skin = False
  has_dress =False
  # Iterate through the 2D tensor array with indices

  # Convert pytorch tensor to numpy array to optimize
  segmentation = pred_seg.detach().cpu().numpy()

  for i in range(rows):
      for j in range(cols):
        v = segmentation[i, j]
        if v == 4: # upper clothes
          upper_mask[i,j] = True
          has_upper = True
        elif v == 6 or v == 5: # pants or skirt
          lower_mask[i,j]=  True
          has_lower = True
        elif v in [11,12,13,14,15]: #body parts
          skin_mask[i,j]=  True
          has_skin = True
        elif v== 7: # dress:
          dress_mask[i,j]=  True
          has_dress = True


  if not has_skin:
    return None
  if has_dress:
    dress_color = getColorDominantFromMask(image, dress_mask)
    return (dress_color, dress_color,getColorDominantFromMask(image, skin_mask) )
  elif not has_upper or not has_lower:
    return None
  else:
    return (getColorDominantFromMask(image, upper_mask),getColorDominantFromMask(image, lower_mask), getColorDominantFromMask(image, skin_mask))


def get_name(name):
  return image.filename.split("-",2)[2].split('.')[0].split('_',1)[1]

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
processor = SegformerImageProcessor.from_pretrained("mattmdjaga/segformer_b2_clothes")
model = AutoModelForSemanticSegmentation.from_pretrained("mattmdjaga/segformer_b2_clothes")
device = "cuda:0" if torch.cuda.is_available() else "cpu"
model.to(device)

file_result = open('/content/drive/MyDrive/men.csv', 'w')
unzip_dir = "men_compressed"
i = 0
for filename in os.listdir(unzip_dir):
    f = os.path.join(unzip_dir, filename)
    image = Image.open(f)
    print("Processing", i, f)
    color = get_colors_men(image)
    if color != None:
      file_result.write(get_name(image.filename) + ';')
      for k in color:
          file_result.write(str(k[0]) + ',' + str(k[1]) + ',' + str(k[2]))
          if k != color[-1]:
            file_result.write(';')
      file_result.write("\n")
    i = i + 1

file_result.close()


In [ ]:
processor = SegformerImageProcessor.from_pretrained("mattmdjaga/segformer_b2_clothes")
model = AutoModelForSemanticSegmentation.from_pretrained("mattmdjaga/segformer_b2_clothes")
device = "cuda:0" if torch.cuda.is_available() else "cpu"
model.to(device)

file_result = open('/content/drive/MyDrive/style_du_monde_2008_2019.csv', 'w')
unzip_dir = "style_du_monde_2008_2019"
i = 0
for filename in os.listdir(unzip_dir):
    f = os.path.join(unzip_dir, filename)
    image = Image.open(f)
    print("Processing", i, f)
    color = get_colors_women(image)
    if color != None:
      file_result.write(image.filename + ';')
      for k in color:
          file_result.write(str(k[0]) + ',' + str(k[1]) + ',' + str(k[2]))
          if k != color[-1]:
            file_result.write(';')
      file_result.write("\n")
    i = i + 1

file_result.close()

Le flux de sortie a été tronqué et ne contient que les 5000 dernières lignes.
(177, 204, 221)
(127, 82, 59)
Processing 3456 style_du_monde_2008_2019/Caroline-Issa-by-STYLEDUMONDE-Street-Style-Fashion-Photography0E2A4265.jpg
(146, 179, 168)
(69, 63, 63)
(138, 90, 50)
Processing 3457 style_du_monde_2008_2019/Kiki-Willems-by-STYLEDUMONDE-Street-Style-Fashion-Photography_MG_7414.jpg
(28, 31, 46)
(15, 15, 23)
(215, 182, 167)
Processing 3458 style_du_monde_2008_2019/Negin-Mirsalehi-by-STYLEDUMONDE-Street-Style-Fashion-Photography-FW18-20180221_48A9943.jpg
(157, 140, 133)
(186, 171, 168)
(90, 49, 31)
Processing 3459 style_du_monde_2008_2019/Natalie-Hartley-by-STYLEDUMONDE-Street-Style-Fashion-Photography0E2A1683.jpg
(24, 28, 40)
(14, 14, 22)
(240, 189, 158)
Processing 3460 style_du_monde_2008_2019/Caroline-Issa-by-STYLEDUMONDE-Street-Style-Fashion-Photography0E2A8577.jpg
(95, 77, 65)
(119, 154, 173)
(103, 76, 59)
Processing 3461 style_du_monde_2008_2019/Giovanna-Engelbert-Battaglia-by-STYLEDU